In [6]:
pip install pymongo==4.7.2 crewai==0.22.5 langchain==0.1.10 langchain-community langchain-openai==0.0.5 duckduckgo-search==6.1.5

  Using cached pymongo-4.7.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (670 kB)
  Using cached crewai-0.22.5-py3-none-any.whl (51 kB)
  Using cached langchain-0.1.10-py3-none-any.whl (806 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 8.7 MB/s eta 0:00:00
  Using cached langchain_openai-0.0.5-py3-none-any.whl (29 kB)
  Using cached duckduckgo_search-6.1.5-py3-none-any.whl (24 kB)
  Using cached dnspython-2.6.1-py3-none-any.whl (307 kB)
  Using cached instructor-0.5.2-py3-none-any.whl (33 kB)
  Using cached openai-1.34.0-py3-none-any.whl (325 kB)
  Using cached opentelemetry_api-1.25.0-py3-none-any.whl (59 kB)
  Using cached opentelemetry_exporter_otlp_proto_http-1.25.0-py3-none-any.whl (16 kB)
  Using cached opentelemetry_sdk-1.25.0-py3-none-any.whl (107 kB)
  Using cached python_dotenv-1.0.0-py3-none-any.whl (19 kB)
  Using cached regex-2023.12.25-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (773 kB)
  Using cached dataclasses_json-0.6.7-py

In [7]:
import os
import pymongo
import pprint

# MongoDB Setup
MDB_URI = "mongodb+srv://<user>:<password>@cluster0.abc123.mongodb.net/"
client = pymongo.MongoClient(MDB_URI)
db = client["sample_analytics"]
collection = db["transactions"]

In [8]:
# Azure OpenAI Setup
from langchain_openai import AzureChatOpenAI
AZURE_OPENAI_ENDPOINT = "https://__DEMO__.openai.azure.com"
AZURE_OPENAI_API_KEY = "__AZURE_OPENAI_API_KEY__"
deployment_name = "gpt-4-32k"  # The name of your model deployment
default_llm = AzureChatOpenAI(
	openai_api_version=os.environ.get("AZURE_OPENAI_VERSION", "2023-07-01-preview"),
	azure_deployment=deployment_name,
	azure_endpoint=AZURE_OPENAI_ENDPOINT,
	api_key=AZURE_OPENAI_API_KEY
)


In [9]:
# Web Search Setup
from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchResults
duck_duck_go = DuckDuckGoSearchResults(backend="news",max_results=10)

In [10]:
# Search Tool - Web Search
@tool
def search_tool(query: str):
  """
  Perform online research on a particular stock.
  Will return search results along with snippets of each result.
  """
  print("\n\nSearching DuckDuckGo for:", query)
  search_results = duck_duck_go.run(query)
  search_results_str =  "[recent news for: " + query + "]\n" + str(search_results)
  return search_results_str

In [11]:
# Research Agent Setup
from crewai import Crew, Process, Task, Agent
AGENT_ROLE = "Investment Researcher"
AGENT_GOAL = """
  Research stock market trends, company news, and analyst reports to identify potential investment opportunities.
"""
researcher = Agent(
  role=AGENT_ROLE,
  goal=AGENT_GOAL,
  verbose=True,
  llm=default_llm,
  backstory='Expert stock researcher with decades of experience.',
  tools=[search_tool]
)

task1 = Task(
  description="""
Using the following information:

[VERIFIED DATA]
{agg_data}

*note*
The data represents the net gain or loss of each stock symbol for each transaction type (buy/sell).
Net gain or loss is a crucial metric used to gauge the profitability or efficiency of an investment.
It's computed by subtracting the total buy value from the total sell value for each stock.
[END VERIFIED DATA]

[TASK]
- Generate a detailed financial report of the VERIFIED DATA.
- Research current events and trends, and provide actionable insights and recommendations.


[report criteria]
  - Use all available information to prepare this final financial report
  - Include a TLDR summary
  - Include 'Actionable Insights'
  - Include 'Strategic Recommendations'
  - Include a 'Other Observations' section
  - Include a 'Conclusion' section
  - IMPORTANT! You are a friendly and helpful financial expert. Always provide the best possible answer using the available information.
[end report criteria]
  """,
  agent=researcher,
  expected_output='concise markdown financial summary of the verified data and list of key points and insights from researching current events',
  tools=[search_tool],
)
# Crew Creation
tech_crew = Crew(
  agents=[researcher],
  tasks=[task1],
  process=Process.sequential
)

In [12]:
# MongoDB Aggregation Pipeline
pipeline = [
  {
    "$unwind": "$transactions"  # Deconstruct the transactions array into separate documents
  },
  {
    "$group": {             		 # Group documents by stock symbol
      "_id": "$transactions.symbol",  # Use symbol as the grouping key
      "buyValue": {           		 # Calculate total buy value
   	 "$sum": {
 		 "$cond": [          		 # Conditional sum based on transaction type
   		 { "$eq": ["$transactions.transaction_code", "buy"] },  # Check for "buy" transactions
   		 { "$toDouble": "$transactions.total" },      		 # Convert total to double for sum
   		 0                                         		 # Default value for non-buy transactions
 		 ]
   	 }
      },
      "sellValue": {          		 # Calculate total sell value (similar to buyValue)
   	 "$sum": {
 		 "$cond": [
   		 { "$eq": ["$transactions.transaction_code", "sell"] },
   		 { "$toDouble": "$transactions.total" },
   		 0
 		 ]
   	 }
      }
    }
  },
  {
    "$project": {            		 # Project desired fields (renaming and calculating net gain)
      "_id": 0,               		 # Exclude original _id field
      "symbol": "$_id",        		 # Rename _id to symbol for clarity
      "netGain": { "$subtract": ["$sellValue", "$buyValue"] }  # Calculate net gain
    }
  },
  {
    "$sort": { "netGain": -1 }  # Sort results by net gain (descending)
  },
  {"$limit": 3}  # Limit results to top 3 stocks
]
results = list(collection.aggregate(pipeline))
client.close()

# Print MongoDB Aggregation Pipeline Results
print("MongoDB Aggregation Pipeline Results:")

pprint.pprint(results) #pprint is used to  to “pretty-print” arbitrary Python data structures

MongoDB Aggregation Pipeline Results:
[{'netGain': 72769230.71428967, 'symbol': 'amzn'},
 {'netGain': 39912931.04990542, 'symbol': 'sap'},
 {'netGain': 25738882.292086124, 'symbol': 'aapl'}]


In [13]:
# Start the task execution
tech_crew.kickoff(inputs={'agg_data': str(results)})



> Entering new CrewAgentExecutor chain...
Given the net gain data of the stocks, the first step would be to analyze the performance of each stock. This can be done by comparing the net gain of each stock. The next step would be to research recent news and trends about these stocks to provide actionable insights and recommendations. Let's start by researching each stock separately. 

Action: search_tool
Action Input: {"query": "amzn stock news"}

Searching DuckDuckGo for: amzn stock news
 

[recent news for: amzn stock news]
[snippet: Jeff Bezos' Amazon is one of the true-blue Silicon Valley Magnificent 7 stocks, a 30-year-old company dating all the way back to 1994. And just like all the other Magnificent 7 stocks, it's had an interesting 2024 so far., title: Amazon (AMZN) stock price to reach over $300? 2030 predictions, link: https://www.msn.com/en-us/money/markets/amazon-amzn-stock-price-to-reach-over-300-2030-predictions/ar-BB1oqmvu, date: 2024-06-18T08:14:55+00:00, source: invez

"# Financial Report\n\n## TLDR Summary\n\n- Amazon (AMZN) has the highest net gain among the three stocks, with a net gain of $72,769,230.71. The company remains a strong player in the e-commerce scene. Predictions suggest that the stock price might reach over $300 by 2030.\n- SAP has the second highest net gain with $39,912,931.04. The company recently acquired WalkMe, which could potentially boost its capabilities in the digital adoption field.\n- Apple (AAPL) has the lowest net gain among the three stocks, with a net gain of $25,738,882.29. Despite multiple positive catalysts, recent news indicates potential decline in the AAPL stock price due to a shift in ETF investments.\n\n## Actionable Insights\n\n- The predicted long-term growth of Amazon's stock price suggests investors can consider AMZN for long-term investments.\n- SAP's recent acquisition of WalkMe may enhance its offerings, making it a potential investment for those interested in digital adoption platforms.\n- Apple's fut